In [1]:
import sys
import os
import tensorflow as tf
print("TensorFlow version is "+ tf.__version__)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

TensorFlow version is 2.0.0


In [2]:
# Preparing MNIST Dataset

# Get MNIST dataset
mnist = tf.keras.datasets.mnist
# Load its data into training and test vectors
(x_train, y_train),(x_test, y_test) = mnist.load_data()
# Normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0
# Collect feature size info
imgSize0=len(x_train[0])
imgSize1=len(x_train[0][0])
numPixels=imgSize0*imgSize1
numTrainImages=len(x_train)
featureShape=(1,imgSize0,imgSize1)

print("Training dataset has "+str(numTrainImages))
print("Testing dataset has "+str(len(x_test)))
print("Feature shape is "+str(featureShape))

Training dataset has 60000
Testing dataset has 10000
Feature shape is (1, 28, 28)


In [3]:
# Clear up everything before running
tf.keras.backend.clear_session()

print ('creating a new model')
# Clearup everything before running
tf.keras.backend.clear_session()

# Create model
model = tf.keras.models.Sequential()

# Add layers
# The first layer MUST have input_shape. See the observation above.
model.add(tf.keras.layers.Flatten(input_shape=(28, 28, 1)))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

# Build model and print summary
model.build(input_shape=featureShape)
model.summary()

creating a new model
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 8)                 6280      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                90        
Total params: 6,370
Trainable params: 6,370
Non-trainable params: 0
_________________________________________________________________


In [4]:
# Train model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
print("Training Model")
# NOTE: Increase the number of epochs if you wish better accuracy
history = model.fit(x_train, y_train, epochs=15)

# Loss plot
plt.plot(history.history["loss"], color='r')
plt.title("Loss")
plt.xlabel("Epoch")
plt.show()

# Accuracy plot
plt.plot(history.history["accuracy"], color='b')
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.show()


Training Model


ValueError: Error when checking input: expected flatten_input to have 4 dimensions, but got array with shape (60000, 28, 28)

In [5]:
# Evaluate Keras model performance
keras_test_loss, keras_test_acc = model.evaluate(x_test, y_test)

print('Test loss:', keras_test_loss)
print('Test accuracy:', keras_test_acc)

313/313 [==============================] - 0s 1ms/step - loss: 0.2663 - accuracy: 0.9264
Test loss: 0.26631611585617065
Test accuracy: 0.9264000058174133


In [6]:
# Update saved model
# Note: This is using an older model format
model.save('./mnist-model.h5')

## Keras to ONNX

In [7]:
import keras2onnx
print("keras2onnx Version: "+ keras2onnx.__version__)

# Convert to onnx model
onnx_model = keras2onnx.convert_keras(model, 'mnist-onnx_2', debug_mode=1)
output_model_path = "./mnist-model_2.onnx"

# Save the model in ONNX format
keras2onnx.save_model(onnx_model, output_model_path)

tf executing eager_mode: True
tf.keras model eager_mode: False
Processing a keras layer - (dense_1: <class 'tensorflow.python.keras.layers.core.Dense'>)
	output: dense_1/Identity:0
	input : dense/Identity:0
Processing a keras layer - (dense: <class 'tensorflow.python.keras.layers.core.Dense'>)
	output: dense/Identity:0
	input : flatten/Identity:0
Processing a keras layer - (flatten: <class 'tensorflow.python.keras.layers.core.Flatten'>)
	output: flatten/Identity:0
	input : flatten_input:0
var: flatten_input
var: flatten_input:0
var: flatten_input:01
var: flatten/Identity:0
var: dense/Identity:0
var: dense_1/Identity:01
var: dense_1/Identity:0
var: dense_1
Converting the operator (Identity): Identity
Converting the operator (Identity1): Identity
Converting the operator (Identity2): Identity
Converting the operator (dense_1): <class 'tensorflow.python.keras.layers.core.Dense'>
Converting the operator (dense): <class 'tensorflow.python.keras.layers.core.Dense'>
Converting the operator (fl

keras2onnx Version: 1.7.0
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 8)                 6280      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                90        
Total params: 6,370
Trainable params: 6,370
Non-trainable params: 0
_________________________________________________________________
None


## ONNX to Keras

In [9]:
import onnx
from onnx2keras import onnx_to_keras

# Load ONNX model
onnx_model = onnx.load('mnist-model_2.onnx')

# Call the converter (input - is the main model input name, can be different for your model)
k_model = onnx_to_keras(onnx_model, ['flatten_input'])

INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> flatten_input.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> dense_1.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight dense_1/kernel:0 with shape (8, 10).
DEBUG:onnx2keras:Found weight dense_1/bias:0 with shape (10,).
DEBUG:onnx2keras:Found weight dense/kernel:0 with shape (784, 8).
DEBUG:onnx2keras:Found weight dense/bias:0 with shape (8,).
DEBUG:onnx2keras:Found input flatten_input with shape [28, 28, 1]
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Identity
DEBUG:onnx2keras:node_name: flatten_input:0
DEBUG:onnx2keras:node_params: {'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name flatten_input).
DEBUG:onnx2keras:... fo

AttributeError: Current node is not in weights / model inputs / layers.